
# Global Terrorism Analysis — Yearly Incident Count Forecasting
**Ready-to-run Jupyter Notebook**

This notebook walks through:
- Loading the Global Terrorism Database (GTD) CSV (download from Kaggle and place in the working folder)
- Cleaning & aggregating to yearly incident counts
- Visualizing trends
- Forecasting with **ARIMA**, **Prophet**, and **LSTM** (optional deep-learning)
- Evaluating model performance and plotting future forecasts

**How to use**
1. Download the GTD CSV (`globalterrorismdb.csv`) from the Kaggle dataset you provided and place it in the same folder as this notebook, or update the `DATA_PATH` variable below.
2. Run all cells in order. Required packages will be installed in the first cell if missing.


In [ ]:

# Install required packages (run once). Uncomment if running in a fresh environment.
import sys
import subprocess

def install(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

packages = [
    "pandas",
    "matplotlib",
    "seaborn",
    "scikit-learn",
    "statsmodels",
    "numpy",
    "prophet",        # note: package name is 'prophet' for recent versions
    "tensorflow",
    "keras",
    "tqdm"
]

# If you're running on Colab you might want to use a fresh kernel after installing.
for p in packages:
    try:
        __import__(p.split("==")[0])
    except Exception as e:
        print(f"Installing {p}...")
        install(p)
print("Done. If installations occurred, restart the kernel and run again.")

In [ ]:

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12,6)

# Set this to the CSV file path you downloaded from Kaggle
DATA_PATH = "globalterrorismdb.csv"  # <<-- change if your file has a different name or path

# Quick check
print('Working directory:', os.getcwd())
print('Looking for data file at DATA_PATH =', DATA_PATH)
assert os.path.exists(DATA_PATH), f"Data file not found at {DATA_PATH}. Please download GTD CSV and update DATA_PATH."

In [ ]:

# Load dataset (the GTD CSV is large; set low_memory False to avoid dtype warnings)
df = pd.read_csv(DATA_PATH, encoding='ISO-8859-1', low_memory=False)
print('Rows, columns:', df.shape)
df.head()

In [ ]:

# The GTD uses 'iyear' for incident year. Some datasets may include placeholder or missing values.
# We'll aggregate incident *rows* per year (one row = one incident record).
# First examine iyear distribution and problematic values:
print('Unique iyear examples:', sorted(df['iyear'].dropna().unique())[:10])

# Group by year (drop rows where iyear is missing or invalid)
yearly = df[df['iyear'].notna()].groupby('iyear').size().reset_index(name='attacks')
yearly = yearly.sort_values('iyear').set_index('iyear')
yearly.index = pd.to_datetime(yearly.index.astype(int), format='%Y')
yearly = yearly.asfreq('Y')  # ensures continuous yearly index

# Fill any missing years with 0 (if dataset has no incidents recorded for that year)
yearly['attacks'] = yearly['attacks'].fillna(0).astype(int)

print(yearly.head(15))
plt.plot(yearly.index.year, yearly['attacks'], marker='o')
plt.title('Global Terrorist Attacks per Year — Raw Counts')
plt.xlabel('Year')
plt.ylabel('Number of Recorded Incidents')
plt.show()

In [ ]:

# We'll hold out the last 5 years for evaluation by default
HOLDOUT_YEARS = 5
train = yearly.iloc[:-HOLDOUT_YEARS]
test = yearly.iloc[-HOLDOUT_YEARS:]

print('Train years:', train.index.year.min(), '-', train.index.year.max())
print('Test years :', test.index.year.min(), '-', test.index.year.max())

plt.plot(train.index.year, train['attacks'], label='Train', marker='o')
plt.plot(test.index.year, test['attacks'], label='Test', marker='o')
plt.legend()
plt.title('Train / Test split (yearly counts)')
plt.show()

In [ ]:

# ARIMA forecasting
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA

series = train['attacks']

# ADF test for stationarity
adf_res = adfuller(series)
print('ADF statistic:', adf_res[0])
print('p-value:', adf_res[1])

# We'll try a simple differencing if non-stationary
d = 0 if adf_res[1] < 0.05 else 1
print('Using d =', d)

# Choose p and q using a simple grid search (small range to keep compute light)
best_aic = np.inf
best_order = None
best_m = None
for p in range(0,3):
    for q in range(0,3):
        try:
            model = ARIMA(series, order=(p,d,q)).fit(method='innovations_mle')
            if model.aic < best_aic:
                best_aic = model.aic
                best_order = (p,d,q)
                best_m = model
        except Exception as e:
            continue

print('Best ARIMA order:', best_order, 'AIC:', best_aic)
arima_model = best_m
# Forecast for test period + future 10 years
steps = HOLDOUT_YEARS + 10
arima_forecast = arima_model.forecast(steps=steps)
arima_forecast_index = pd.date_range(start=train.index[-1] + pd.offsets.YearEnd(1), periods=steps, freq='Y')
arima_forecast = pd.Series(arima_forecast, index=arima_forecast_index)

# Evaluate on test portion
arima_pred_test = arima_forecast[:HOLDOUT_YEARS].values
rmse_arima = sqrt(mean_squared_error(test['attacks'].values, arima_pred_test))
mae_arima = mean_absolute_error(test['attacks'].values, arima_pred_test)
print('ARIMA RMSE on test:', rmse_arima, 'MAE:', mae_arima)

# Plot
plt.plot(yearly.index.year, yearly['attacks'], label='Actual', marker='o')
plt.plot(arima_forecast_index.year, arima_forecast.values, label='ARIMA Forecast', linestyle='--')
plt.axvline(x=test.index.year.min()-0.5, color='gray', alpha=0.5, linestyle=':')
plt.title('ARIMA Forecast (including test & future)')
plt.legend()
plt.show()

In [ ]:

# Prophet forecasting
try:
    from prophet import Prophet
except Exception as e:
    raise ImportError('Prophet not available. Install `prophet` package and restart the kernel.') from e

prophet_df = yearly.reset_index().rename(columns={'iyear':'ds','attacks':'y'})
prophet_df['ds'] = pd.to_datetime(prophet_df['ds'].dt.year.astype(str) + '-12-31')  # use year-end as ds for yearly freq

m = Prophet(yearly_seasonality=True)
m.fit(prophet_df[:-HOLDOUT_YEARS])  # fit on train portion

future = m.make_future_dataframe(periods=HOLDOUT_YEARS+10, freq='Y')
forecast = m.predict(future)

# Extract forecast series and align index
prophet_forecast = forecast.set_index('ds')['yhat']
prophet_forecast.index = pd.to_datetime(prophet_forecast.index).to_period('Y').to_timestamp('A')  # year-end timestamps

# Evaluate on test set (align years)
prophet_pred_test = prophet_forecast[-(HOLDOUT_YEARS+10):-10].values  # first HOLDOUT_YEARS of future portion
rmse_prophet = sqrt(mean_squared_error(test['attacks'].values, prophet_pred_test))
mae_prophet = mean_absolute_error(test['attacks'].values, prophet_pred_test)
print('Prophet RMSE on test:', rmse_prophet, 'MAE:', mae_prophet)

# Plot prophet forecast
m.plot(forecast)
plt.title('Prophet Forecast (includes train + forecast)')
plt.show()

In [ ]:

# LSTM forecasting (sequence model) - optional and more advanced.
# We'll build a simple LSTM on the scaled attack counts.
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

series_values = yearly['attacks'].values.astype('float32')

scaler = MinMaxScaler(feature_range=(0,1))
scaled = scaler.fit_transform(series_values.reshape(-1,1)).flatten()

# helper to make supervised sequences
def create_sequences(data, seq_len=5):
    X, y = [], []
    for i in range(len(data)-seq_len):
        X.append(data[i:i+seq_len])
        y.append(data[i+seq_len])
    return np.array(X), np.array(y)

SEQ_LEN = 5
X, y = create_sequences(scaled, SEQ_LEN)

# Split to train/test using the same HOLDOUT_YEARS logic (keep temporal order)
train_size = len(yearly) - HOLDOUT_YEARS
X_train, y_train = X[:train_size-SEQ_LEN], y[:train_size-SEQ_LEN]
X_test, y_test = X[train_size-SEQ_LEN:], y[train_size-SEQ_LEN:]

# reshape for LSTM [samples, timesteps, features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

model = Sequential([
    LSTM(32, input_shape=(SEQ_LEN,1)),
    Dense(8, activation='relu'),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
es = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True, verbose=1)

history = model.fit(X_train, y_train, epochs=500, batch_size=8, validation_split=0.1, callbacks=[es], verbose=0)

# Forecast iteratively for HOLDOUT_YEARS + 10 future years
preds = []
last_seq = scaled[-SEQ_LEN:].tolist()
for _ in range(HOLDOUT_YEARS+10):
    inp = np.array(last_seq[-SEQ_LEN:]).reshape(1, SEQ_LEN, 1)
    p = model.predict(inp)[0,0]
    preds.append(p)
    last_seq.append(p)

preds_inv = scaler.inverse_transform(np.array(preds).reshape(-1,1)).flatten()
lstm_index = pd.date_range(start=yearly.index[-1] + pd.offsets.YearEnd(1), periods=HOLDOUT_YEARS+10, freq='Y')
lstm_series = pd.Series(preds_inv, index=lstm_index)

# Evaluate LSTM on test
lstm_pred_test = lstm_series[:HOLDOUT_YEARS].values
rmse_lstm = sqrt(mean_squared_error(test['attacks'].values, lstm_pred_test))
mae_lstm = mean_absolute_error(test['attacks'].values, lstm_pred_test)
print('LSTM RMSE on test:', rmse_lstm, 'MAE:', mae_lstm)

# Plot LSTM forecast
plt.plot(yearly.index.year, yearly['attacks'], label='Actual', marker='o')
plt.plot(lstm_series.index.year, lstm_series.values, label='LSTM Forecast', linestyle='--')
plt.legend(); plt.title('LSTM Forecast (iterative)'); plt.show()


## Model Comparison & Next Steps

- The notebook fits **ARIMA**, **Prophet**, and an **LSTM** model (LSTM is optional but included for completeness).
- Each model provides RMSE/MAE on the held-out years for easy comparison.
- **Next steps / improvements**:
  - Add exogenous regressors (political indicators, GDP, conflict events) for better forecasting.
  - Use cross-validation on time series (walk-forward validation) instead of a single holdout.
  - Improve LSTM with hyperparameter search, or try Transformers for time series (e.g., Temporal Fusion Transformer).
  - Build an interactive dashboard with Streamlit or Plotly Dash to let users choose models and horizons.

**End of notebook.**
